# Population Artificial Neuron

Rabbits multiply quickly, but with limited resources, their population eventually stabilizes. 🐇

In this notebook, your task is to build an artificial neuron that models the population size of rabbits over time.

Your neuron should predict how the rabbit population grows and then levels off as time passes. By the end, your model should be able to approximate the true population curve based on the data provided.

In [32]:
import pandas as pd
import math
import numpy as np
import random

train = pd.read_csv("pushups_vs_muscle.csv")
inputs = train['weekly_hrs_training']
expecteds = train['lean_muscle_pct']

First, it might be a good idea to visualize your data. 

In [33]:
import plotly.express as px


fig = px.line(train, x="weekly_hrs_training", y="lean_muscle_pct", title='Pushups vs Muscle')
fig.show()

Now, create a single input artificial neuron that takes in time as its only input and predicts the rabbit population size.

- This time the input will be passed into the function, so no need to ask for user input.
- Choose an appropriate activation function that allows your model to represent the growth and stabilization pattern.
- Use a **non-zero bias** and a **non-zero weight** to ensure your model can properly fit the data.
- You may need to use trial and error to find the best combination of weight, bias, and activation function.

In [34]:
#base functions

def activation(z):
    return (1 / (1 + np.exp(-z)))

def z_calc(x, weight, bias):
    z = (x * weight) + bias
    return z

def output(x, weight, bias):
    return activation(z_calc(x, weight, bias))

def dw(x, y, weight, bias):
    z = z_calc(x, weight, bias)
    return ((activation(z) - y)*((activation(z))*(1-activation(z)))*(x))

def db(x, y, weight, bias):
    z = z_calc(x, weight, bias)
    return ((activation(z)-y)*((activation(z))*(1-activation(z))))

def MSE(x, y):
    return 0.5 * (x-y) ** 2

Run the code below to see if you implemented your neuron correctly. It will check if your solution is close enough to our data.

In [ ]:
#lvl 2

l_rate = 0.5
weight = 2
bias = 2

parameters = (weight, bias)

def single_round(inputs, targets, weight, bias, l_rate):
    d_weights = []
    d_bias = []

    for i in range(0, len(inputs)):
        d_weights.append(dw(inputs[i], expecteds[i], weight, bias))
        d_bias.append(db(inputs[i], expecteds[i], weight, bias))
    
    new_weight = weight - l_rate * (sum(d_weights)/len(d_weights))
    new_bias = bias - l_rate * (sum(d_bias)/len(d_bias))

    return (new_weight, new_bias)

def calc_loss(inputs, expecteds, weight, bias):
    loss = []

    for i in range(0, len(inputs)):
        loss.append(MSE(output(inputs[i], weight, bias), expecteds[i]))

    return sum(loss)/len(loss)
    
def back_propagation_loop(inputs, expecteds, weight, bias, l_rate, iterations):
    losses = []

    for iteration in range(iterations):
        weight, bias = single_round(inputs, expecteds, weight, bias, l_rate)
        loss = calc_loss(inputs, expecteds, weight, bias)
        losses.append(loss)

    return weight, bias, losses

l_rate = 0.5
weight = random.randint(1,3)
bias = random.randint(1,3)
iterations = 5000

final_w, final_b, losses = back_propagation_loop(inputs, expecteds, weight, bias, l_rate, iterations)

print("Final Weight:", final_w)
print("Final Bias:", final_b)

fig2 = px.line(y=losses, title="Training Loss Over Iterations")
fig2.update_layout(xaxis_title="Iteration", yaxis_title="Loss")
fig2.show()

How does learning rate affect the number of iterations you need in order to reach convergence?: Increasing the learning rate allows the loop to reach convergence faster as each jump goes farther.
What happens to your loss curve if your learning rate is too high?: The high learning rate will casue the loop to overshoot the convergence point and oscilate around it.
What is the maximum range that you can choose your initial weight and bias from? Why is this?: For the sigmoid function, if w and b are too big, z will also be big, and the sigmoid could saturate to 1 or 0, stalling loss progression.
